In [ ]:
# This is a demonstration of basic use cases for the Code Ocean SDK.

In [ ]:
from codeocean import CodeOcean
from codeocean.computation import RunParams, DataAssetsRunParam, PipelineProcessParams
from codeocean.data_asset import (
    DataAssetParams,
    Source,
    ComputationSource,
    AWSS3Source,
    Target,
    AWSS3Target
)

In [ ]:
# Create the client using your domain and API token. If not available, you can generate one.

client = CodeOcean(domain='https://{your_domain}.codeocean.com', token='cop_ZDky')

In [ ]:
# Creating an Internal Data Asset from Public AWS S3 

data_asset_params = DataAssetParams(
        name="Dataset From SDK",
        description="This Data Asset is a Public Asset",
        mount="References",
        tags=["Bioinformatics","RNA"],
        source=Source(
            aws=AWSS3Source(
                bucket="codeocean-public-data",
                prefix=f"unit_test/Reads/Sample_{number}/",
                public=True,
                keep_on_external_storage=False
                )))

data_asset = client.data_assets.create_data_asset(data_asset_params)

data_asset = client.data_assets.wait_until_ready(data_asset)

In [ ]:
# Manually - Create a New Pipeline - Add the Data Asset from S3 to the Pipeline - Drag onto the UI 
# Drag FastQC and MultiQC onto the UI and Attach the Data Asset to FastQC - Set Connection Type to Default.
# FastQC will be connected to MultiQC - Set Connection Type to Collect.
# MultiQC will be connected to the Results Bucket. 

In [ ]:
data_assets=[
        DataAssetsRunParam(id=data_asset.id,
                              mount="Reads")
        ]    
    
processes=[
          PipelineProcessParams(
                    name="capsule_fastqc", # This name can be found in /pipeline/main.nf - search for fastQC
                    parameters=[
                        "1"
                        ]
                  ),
        PipelineProcessParams(
                    name="capsule_multiqc", # This name can be found in /pipeline/main.nf - search for multiqc
                    parameters=[
                        "myReport"
                        ]
                  ),
        ]
        
run_params = RunParams(
                    pipeline_id="4bc97533-6eb4-48ac-966f-648548a756d2",
                    data_assets=data_assets,
                    processes=processes    
               )

computation = client.computations.run_capsule(run_params)

# Wait for Pipeline to finish.

computation = client.computations.wait_until_completed(computation)